#### AFSK Demodulator
## Digital PLL

This is a Pynq portion of the AFSK demodulator project.  We will be using the FPGA overlay that we created in Vivado.

At this point we have created the bitstream for the Digital PLL and copied the bitstream, TCL wrapper, and hardware hand-off file to the Pynq board.

Let's first verify that we can load the module.

In [1]:
from pynq import Overlay, Xlnk
import numpy as np
import pynq.lib.dma

overlay = Overlay('digital_pll.bit')
dma = overlay.demodulator.dma
pll = overlay.demodulator.pll
print(pll.register_map)

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0)
}


## Implementing the Digital PLL

Below is the implementation of the AFSK demodulator in Python.  We are now going to remove the digital PLL code and replace it with new code.  The new code has a slightly different interface.  We get back three bits that we need to unpack into Boolean values.

In [2]:
import sys
sys.path.append('../../base')

import numpy as np
from scipy.signal import lfiltic, lfilter, firwin
from scipy.io.wavfile import read
from DigitalPLL import DigitalPLL
from HDLC import HDLC
from AX25 import AX25
import time


class fir_filter(object):
    def __init__(self, coeffs):
        self.coeffs = coeffs
        self.zl = lfiltic(self.coeffs, 32768.0, [], [])
    def __call__(self, data):
        result, self.zl = lfilter(self.coeffs, 32768.0, data, -1, self.zl)
        return result

class NRZI:

    def __init__(self):

        self.state = False

    def __call__(self, x):
        
        result = (x == self.state)
        self.state = x
        return result

audio_file = read('../../base/TNC_Test_Ver-1.101-26400-1min.wav')
sample_rate = audio_file[0]
audio_data = audio_file[1]
delay = 12 # ~446us

bpf_coeffs = np.array(firwin(141, [1100.0/(sample_rate/2),2300.0/(sample_rate/2)], width = None,
        pass_zero = False, scale = True, window='hann') * 32768, dtype=int)
lpf_coeffs = np.array(firwin(101, [760.0/(sample_rate/2)], width = None,
        pass_zero = True, scale = True, window='hann') * 32768, dtype=int)

bpf = fir_filter(bpf_coeffs)
lpf = fir_filter(lpf_coeffs)

# filter_delay = 64 + 50

bpf_delay = 70
lpf_delay = 50
filter_delay = bpf_delay + lpf_delay

# Band-pass filter the audio data
print("Doing BPF...")
f = bpf(np.append(audio_data, np.zeros(filter_delay, dtype=int)))
# Digitize the data
print("Digitizing audio data...")
print(len(f))
d = np.greater_equal(f, 0)
print(len(d))
# Delay the data
print("Delay...")
a = d[delay:]
# XOR the digitized data with the delayed version
print("Doing Logical XOR...")
x = np.logical_xor(d[:0-delay], a, dtype=int)
# Low-pass filter the PWM signal
print("Doing LPF...")
c = lpf(x-0.5)
# Digitize the tone transistions
print(len(c))
print("Digitizing correlator output...")
dx = np.greater_equal(c, 0.0)

Doing BPF...
Digitizing audio data...
1584120
1584120
Delay...
Doing Logical XOR...
Doing LPF...
1584108
Digitizing correlator output...


In [4]:
block_size = 26400

xlnk = Xlnk()

def demod(data):
    pll.register_map.CTRL = 0x81 # START|AUTO-RESTART while writing data
    start_time = time.time()
    output = np.array([],dtype=np.bool)
    with xlnk.cma_array(shape=(block_size,), dtype=np.int8) as out_buffer, \
        xlnk.cma_array(shape=(block_size,), dtype=np.int8) as in_buffer:

        for i in range(0, len(data), block_size):
            out_buffer[:len(data[i:i+block_size])] = data[i:i+block_size]
            dma.sendchannel.transfer(out_buffer)
            dma.recvchannel.transfer(in_buffer)
            dma.sendchannel.wait()
            dma.recvchannel.wait()
            output = np.append(output, in_buffer)
            
    stop_time = time.time()
    sw_exec_time = stop_time - start_time
    print('FPGA digital PLL execution time: ',sw_exec_time)
    pll.register_map.CTRL = 0
    return output

dpll = demod(dx)

nrzi = NRZI()
hdlc = HDLC()
count = 0
for x in dpll:
    if bool(x & 2):
        packet = hdlc(nrzi(bool(x & 4)), bool(x & 1))
        if packet is not None:
            count += 1
            print(count, AX25(packet[1]))

print("Done")

FPGA digital PLL execution time:  0.5146293640136719
1 WA6YLB-8>APRS,N6EX-11:$ULTW00000000----0000----000086A00001----0000000000000000???P
2 KD6FVP-4>APS224,N6EX-2,WIDE1-1:>152343z[224]*We know most of your faults!!!?q?
3 KD6FVP-4>APS224,N6EX-2,WIDE1-1:>152343z[224]*We know most of your faults!!!???
4 N6XQY-8>GPSLJ,RELAY,WIDE2-5:$GPRMC,013641.06,A,3348.1607,N,11807.4631,W,34.0,090.5,231105,13.,E*73??B
5 WA6YLB>APRX46,WA6YLB-14,W6SCE-5:>081839z wa6ylb@theworks.com??f
6 KC6HUR-2>S4QVYV,W6SCE-5:'.4&l-/k/]"7q}???
7 N6XQY-8>GPSLJ,N6EX-9:$GPRMC,013641.06,A,3348.1607,N,11807.4631,W,34.0,090.5,231105,13.,E*73???
8 KC6BLF-12>S4PWYS,N6EX-11:'-U l{(u/]"5\}Lost in the West!?;:
9 K6KMA-2>GPSLK,N6EX-3:$GPRMC,013647,A,3350.076,N,11806.996,W,028.3,180.5,231105,013.5,E*69????
10 AE6GR-14>S4PXYW,WIDE2-5:'._|l tv/]"6[}?:c
11 AE6GR-14>S4PXYW,N6EX-3:'._|l tv/]"6[}???
12 AE6MP>SS5PPQ-4,WIDE2-5:`.](n->>/"4W}??
13 AE6MP>SS5PPQ-4,N6EX-9:`.](n->>/"4W}??
14 AE6MP>SS5PPQ-4,N6EX-3:`.](n->>/"4W}??
15 AE6MP>SS5PPQ-2

In [ ]:
ctrl = dma.recvchannel._mmio.read(dma.recvchannel._offset)
print(ctrl)
dma.recvchannel._mmio.write(dma.recvchannel._offset, (ctrl | 4) & 0xFFFFFFFE)
print(dma.recvchannel._mmio.read(dma.recvchannel._offset+0x04))
dma.recvchannel.start()
dma.sendchannel.start()